# Make a simple Convolutional network for the bus data

## Initialization

In [14]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../misc')
sys.path.append("../")
from MoviaBusDataset import MoviaBusDataset
from BaseNetwork import BaseNetwork
import torch
from torch.nn import Linear, Sequential, ReLU, L1Loss
from torch.optim import Adam, lr_scheduler
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import data using the MoviaBusDataset

In [8]:
previous_timesteps = 6
prediction_steps = 6
batch_size = 25
num_epochs = 100

train = MoviaBusDataset('../data/train', interpolation=True, prev_timesteps=previous_timesteps, max_future_time_steps=prediction_steps, timeofday = False)
test = MoviaBusDataset('../data/test', interpolation=True, prev_timesteps=previous_timesteps, max_future_time_steps=prediction_steps, timeofday = False)

In [9]:
train.normalize(individual_roads=False)
test.normalize(train.mean, train.std,individual_roads=False)

## The Model

In [12]:
#from IPython.core.debugger import Tracer; Tracer()()
#%pdb 0

from CNN import CNN, GCN
from FNN import FNN
cnn = CNN(previous_timesteps)

#cnn = GCN(nfeat=previous_timesteps + 1, 
#          nhid=100, 
#          nout=1, 
#          dropout=0.3)



## Training the network

In [13]:
#%pdb 0
cnn.train_network(train, test,batch_size=batch_size, 
                  num_epochs=20,
                  optimizer_fun=lambda param : Adam(param, lr=1e-2, weight_decay=1e-4),
                  scheduler_fun=lambda optim : lr_scheduler.StepLR(optim, step_size=10, gamma=0.1),
                  criterion=L1Loss())

epoch =  0  train loss = 1.024   test loss = 0.851   output_std = 0.047
epoch =  2  train loss = 0.835   test loss = 0.842   output_std = 0.069
epoch =  4  train loss = 0.494   test loss = 0.500   output_std = 0.823
epoch =  6  train loss = 0.489   test loss = 0.496   output_std = 0.809
epoch =  8  train loss = 0.488   test loss = 0.497   output_std = 0.795
epoch = 10  train loss = 0.488   test loss = 0.495   output_std = 0.816
epoch = 12  train loss = 0.486   test loss = 0.497   output_std = 0.843
epoch = 14  train loss = 0.485   test loss = 0.499   output_std = 0.836
epoch = 16  train loss = 0.485   test loss = 0.500   output_std = 0.833
epoch = 18  train loss = 0.485   test loss = 0.500   output_std = 0.829


In [ ]:
fnn = FNN(previous_timesteps = previous_timesteps, num_hidden = 100)
fnn.train_network(train, test,batch_size=batch_size, 
                  num_epochs=30,
                  optimizer_fun=lambda param : Adam(param, lr=1e-2, weight_decay=1e-4),
                  scheduler_fun=lambda optim : lr_scheduler.StepLR(optim, step_size=10, gamma=0.1),
                  criterion=L1Loss())


In [ ]:
a = torch.Tensor(2,4)
print(a.shape)
a = a.unsqueeze(1)
print(a.shape)

## Evaluate the network on a test dataset 

In [ ]:
cnn.get_MAE_score(timestep=6)

### Example of modelled data vs real data for one road segment 

In [ ]:
%matplotlib notebook

cnn.visualize_road(timesteps=1, road=23)